In [4]:
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import requests
from sklearn.multioutput import MultiOutputRegressor, MultiOutputClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_pacf
import itertools
import pandas as pd
from os import path
from pathlib import Path
from sklearn.preprocessing import StandardScaler

PROJECT_DIR = Path(Path.cwd())
DATA_FOLDER = PROJECT_DIR / 'data/'

source = 'polygon'

c:\Users\ma.orlandi\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
c:\Users\ma.orlandi\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [5]:
if source == 'polygon':
    API = 't9Hb6jZUizreYPBZWsAFbm8Imn0zdQZU'
    ticker = 'X:BTCUSD' #symbol
    from_ = '2018-01-01'
    to_ = datetime.today().strftime('%Y-%m-%d')
    market = 'crypto'
    timeframe = 'day'
    depth = 10
    # URL for all the tickers on Polygon
    POLYGON_TICKERS_URL = f'https://api.polygon.io/v3/reference/tickers?market={market}&active=true&sort=ticker&order=asc&apiKey={API}'
    # URL FOR PRICING DATA - Note, getting pricing that is UNADJUSTED for splits, I will try and adjust those manually
    POLYGON_AGGS_URL = f'https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/{timeframe}/{from_}/{to_}?unadjusted=true&apiKey={API}'
    # URL FOR DIVIDEND DATA
    POLYGON_DIV_URL = 'https://api.polygon.io/v2/reference/dividends/{}?apiKey={}'
    # URL FOR STOCK SPLITS
    POLYGON_SPLIT_URL = 'https://api.polygon.io/v2/reference/splits/{}?apiKey={}'
    #URL FOR TICKER TYPES
    POLYGON_TYPES_URL = 'https://api.polygon.io/v2/reference/types?apiKey={}'
elif source == 'twelvedata':
    API="7574f9ef4be7496e976c0d1652415f34"
    ticker="BTC/USD"
    timeframe="1day"
    outputsize=5000
    timezone="America/New_York"
    howmanyrows=5000
    n_test=50
elif source == 'binance':
    market = 'BTCUSDT'
    timeframe = '1h'
    url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval
    col_names = ['OpenTimestamp','Open','High','Low','Close','Volume','CloseTimestamp','Quote asset volume','Number of trades','Taker buy base asset volume','Taker buy quote asset volume','Ignore.']
plot = False



In [6]:
#from twelvedata
def get_historicaldata(mykey, symbol, interval, outputsize,timezone) -> TDClient:

    # Initialize client - apikey parameter is requiered
    td = TDClient(apikey=mykey)
    # Construct the necessary time serie
    ts = td.time_series(
        symbol=symbol,
        interval=interval,
        outputsize=outputsize,
        timezone=timezone,
    )

    #return ts.with_bbands(ma_type="EMA").with_plus_di().with_wma(time_period=20).with_wma(time_period=40)
    return ts

NameError: name 'TDClient' is not defined

In [7]:
def load_data(apikey:str, project_path:str, symbol: str, interval: str, outputsize: str, timezone: str, n: int, overwrite: bool=False) -> pd.DataFrame:
    
    data_path = project_path / (symbol.replace("/","_").upper() + '_' + interval + '.csv')

    if not path.exists(data_path):
        df = get_historicaldata(apikey, symbol, interval, outputsize, timezone).as_pandas()
        df.reset_index(level=0, inplace=True)
        df.to_csv(data_path)
    else:
        if overwrite:
            df = get_historicaldata(apikey, symbol, interval, outputsize, timezone).as_pandas()
            df.to_csv(data_path)
        else:
            df = pd.read_csv(data_path, parse_dates=True, nrows=n)

    return df

In [8]:
def ts_to_datetime(ts) -> str:
    return datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M')

In [9]:
def plt_df(df, _list, plt_size):
    
    figure(figsize=(16, 9))
    for l in _list:
        plt.plot(df['Timestamp'][:plt_size], df[l][:plt_size])
    
    plt.legend(loc='best')
    plt.show()

In [10]:
def apply_indicator(df:pd.DataFrame, ind_type:str, period:list=[], el_list:list=[]) -> pd.DataFrame:  
           
    if not el_list:
        on_elements = [x for x in df.columns if x != "Timestamp"]
    else:
        on_elements = el_list
    
    if ind_type == 'SMA':
        if type(period) is list:
            for p in period:
                for element in on_elements:
                    df["SMA"+ str(p) + "_" + element] = df[element].rolling(p).mean()
    elif ind_type == 'EMA':
        if type(period) is list:
            for p in period:
                for element in on_elements:
                    df["EMA"+ str(p) + "_" + element] = \
                    df[element].ewm(span=p,min_periods=0,adjust=False,ignore_na=False).mean()
    elif ind_type == 'STD':
        if type(period) is list:
            for p in period:
                for element in on_elements:
                    df["STD"+ str(p) + "_" + element] = df[element].rolling(p).std(ddof=0)
    elif ind_type == 'MAX':
        if type(period) is list:
            for p in period:
                for element in on_elements:
                    df["MAX"+ str(p) + "_" + element] = df[element].rolling(p).max()
    elif ind_type == 'MIN':
        if type(period) is list:
            for p in period:
                for element in on_elements:
                    df["MIN"+ str(p) + "_" + element] = df[element].rolling(p).min()
    elif ind_type == 'PCT':                
        df["Pct_change"] = (100*(df["Close"]-df["Close"].shift(+1))/df["Close"]).round(2)
    
    return df

In [11]:
def labelize_output(df: pd.DataFrame, target:list, wrt:str='Close', threshold:float=0.01, risk_reward_ratio:float=0.8) -> pd.DataFrame:
    
    df_ = df.copy()
    positive_criterion = 1 + threshold/risk_reward_ratio
    negative_criterion = 1 - threshold
     
    for column in target:
        conditions = [
            (df_[column] >= df_[wrt] * positive_criterion),
            (df_[column] <= df_[wrt] * negative_criterion)]
        choices = [2, 1]
        df_[f"target_{column}"] = np.select(conditions, choices, default=0)
        df_.drop(column,1,inplace=True)
    return df_
    

In [12]:
def TFs_to_predict(df:pd.DataFrame, apply_on:str, TimeFrame:str, howmanyTFs:int) -> pd.DataFrame:
    #apply_one = any('Open','Close','High','Low')
    for i in range(1, howmanyTFs+1):
        df[f"{TimeFrame}{apply_on}{i}"] = df["Close"].shift(-i)
    
    return df

In [13]:
def split_data(df:pd.DataFrame, labels: list=[]) -> [pd.DataFrame, pd.DataFrame]:
    
    return df.drop(labels,axis=1), df[labels]

In [14]:
def df_train_test_split(dataset: pd.DataFrame, lbls_to_pred: list=[], pct_train=0.8) -> [pd.DataFrame,pd.DataFrame,pd.DataFrame,pd.DataFrame]:
    
    if not lbls_to_pred:
        target = [x for x in dataset.columns if x.startswith('target')]
    else:
        target = lbls_to_pred
 
    X, y = split_data(dataset,target)
    print(X.shape,y.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=1-pct_train)
    
    '''    ts_cv = TimeSeriesSplit(
            n_splits=5,
            gap=0,
            max_train_size=136,
            test_size=10,
        )
    '''    
    return X_train, X_test, y_train, y_test
    

In [15]:
def model_selection(model_type, parameter, objective='regression'):
    
    dict_model = {
        'regression':{
            'xgb': xgb.XGBRegressor(n_jobs=-1, random_state=42),
            'rf': RandomForestRegressor(n_jobs=-1, random_state=42),
            'lgb': LGBMRegressor(n_jobs=-1, random_state=42)
        },
        'classification':{
            'xgb': xgb.XGBClassifier(n_jobs=-1, random_state=42),
            'rf': RandomForestClassifier(n_jobs=-1, random_state=42),
            'lgb': LGBMClassifier(n_jobs=-1, random_state=42)
        }
    }
    
    return dict_model[objective][model_type].set_params(**parameter)

In [16]:
def standard_scaler(array):
    
    scaler = MinMaxScaler().fit(array)
    new_array = scaler.transform(array)
    
    return scaler, new_array

In [17]:
if source == 'polygon':
    data = requests.get(POLYGON_AGGS_URL).json()
    data = data.reset_index(level=0)
elif source == 'binance':
    market = 'BTCUSDT'
    timeframe = '1day'
    url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval
    data = requests.get(url).json()
else:
    data = load_data(API, DATA_FOLDER, ticker, timeframe, outputsize, timezone, howmanyrows)
    
#data['results']

AttributeError: 'dict' object has no attribute 'reset_index'

In [18]:
##### POLYGON
df = pd.DataFrame(data['results'])
df['t'] = df['t'].apply(ts_to_datetime) 
#col_names = ['Volume', 'VolWeightedAVGPrice', 'Open','Close','High', 'Low', 'Timestamp','N_Transact']
df.columns = col_names
#df['MarketCap'] = df['N_Transact'] * df['VolWeightedAVGPrice']
df['year'] = pd.to_datetime(df['Timestamp']).dt.year
df['month'] = pd.to_datetime(df['Timestamp']).dt.month
df['day'] = pd.to_datetime(df['Timestamp']).dt.day
df.drop('Timestamp', 1, inplace=True)
df.head(5)

NameError: name 'col_names' is not defined

In [151]:
df = pd.DataFrame(data)
#col_names = ['Volume', 'VolWeightedAVGPrice', 'Open','Close','High', 'Low', 'Timestamp','N_Transact']
df.columns = col_names
df['OpenTimestamp'] = df['OpenTimestamp'].apply(ts_to_datetime) 
#df['MarketCap'] = df['N_Transact'] * df['VolWeightedAVGPrice']
df['year'] = pd.to_datetime(df['OpenTimestamp']).dt.year
df['month'] = pd.to_datetime(df['OpenTimestamp']).dt.month
df['day'] = pd.to_datetime(df['OpenTimestamp']).dt.day
df.drop('OpenTimestamp', 1, inplace=True)
df.drop('CloseTimestamp', 1, inplace=True)
df.drop('Ignore.', 1, inplace=True)
df.tail(5)

,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,year,month,day
495,30293.93000000,30670.51000000,28866.35000000,29109.15000000,63901.49932000,1914305818.07451870,1328317,30476.66811000,913813980.95331840,2022,5,23
496,29109.14000000,29845.86000000,28669.00000000,29654.58000000,59442.96036000,1740257602.30676980,946421,29480.07674000,863516681.51292810,2022,5,24
497,29654.58000000,30223.74000000,29294.21000000,29542.15000000,59537.38659000,1771708175.07189140,985440,28846.96589000,858587338.50383790,2022,5,25
498,29542.14000000,29886.64000000,28019.56000000,29201.35000000,94581.65463000,2759055410.39775190,1431112,45206.85416000,1319426335.71623620,2022,5,26
499,29201.35000000,29390.34000000,28643.89000000,29005.51000000,47532.81700000,1375670049.12549450,632753,23517.16540000,680697647.65774550,2022,5,27


In [152]:
for col in df.columns:
    if df[col].dtype == 'O':
        print(col)
        df[col] = pd.to_numeric(df[col])
df.dtypes

Open
High
Low
Close
Volume
Quote asset volume
Taker buy base asset volume
Taker buy quote asset volume


Open                            float64
High                            float64
Low                             float64
Close                           float64
Volume                          float64
Quote asset volume              float64
Number of trades                  int64
Taker buy base asset volume     float64
Taker buy quote asset volume    float64
year                              int64
month                             int64
day                               int64
dtype: object

In [153]:
df = apply_indicator(df, 'EMA', [5,10,60,223,7,14,21,28], ['Open','Close','High', 'Low'])
df = apply_indicator(df, 'SMA', [5,10,60,223,7,14,21,28], ['Open','Close','High', 'Low'])
df = apply_indicator(df, 'STD', [5,10,60,223,7,14,21,28], ['Open','Close','High', 'Low'])
df = apply_indicator(df, 'MAX', [5,10,60,223,7,14,21,28], ['Open','Close','High', 'Low'])
df = apply_indicator(df, 'MIN', [5,10,60,223,7,14,21,28], ['Open','Close','High', 'Low'])
df = apply_indicator(df, 'PCT', _, ['Close'])
df

,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,year,...,MIN14_Low,MIN21_Open,MIN21_Close,MIN21_High,MIN21_Low,MIN28_Open,MIN28_Close,MIN28_High,MIN28_Low,Pct_change
0,34049.15,37850.00,32380.00,37371.38,124477.914938,4.322877e+09,2514289,63981.038306,2.222873e+09,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,37371.38,40100.00,36701.23,39144.50,102950.389421,3.974827e+09,2297466,51264.467092,1.979443e+09,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.53
2,39145.21,39747.76,34408.00,36742.22,118300.920916,4.371589e+09,2560801,57738.034220,2.133724e+09,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.54
3,36737.43,37950.00,35357.80,35994.98,86348.431508,3.178840e+09,2066363,42420.423736,1.563160e+09,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.08
4,35994.98,36852.50,33850.00,35828.61,80157.727384,2.843103e+09,1860642,39513.638561,1.402649e+09,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,30293.93,30670.51,28866.35,29109.15,63901.499320,1.914306e+09,1328317,30476.668110,9.138140e+08,2022,...,26700.00,28715.33,28715.32,29656.18,26700.0,28715.33,28715.32,29656.18,26700.0,-4.07
496,29109.14,29845.86,28669.00,29654.58,59442.960360,1.740258e+09,946421,29480.076740,8.635167e+08,2022,...,26700.00,28715.33,28715.32,29656.18,26700.0,28715.33,28715.32,29656.18,26700.0,1.84
497,29654.58,30223.74,29294.21,29542.15,59537.386590,1.771708e+09,985440,28846.965890,8.585873e+08,2022,...,26700.00,28715.33,28715.32,29656.18,26700.0,28715.33,28715.32,29656.18,26700.0,-0.38
498,29542.14,29886.64,28019.56,29201.35,94581.654630,2.759055e+09,1431112,45206.854160,1.319426e+09,2022,...,28019.56,28715.33,28715.32,29656.18,26700.0,28715.33,28715.32,29656.18,26700.0,-1.17


In [108]:
def explore_minima(_df:pd.DataFrame, period:int, _on='Close') -> pd.DataFrame:
    #local minima
    df = _df.copy()
    df['local_max+-'+ str(period)] = df[_on][
      (df[_on].shift(period) < df[_on]) &
      (df[_on].shift(-period) < df[_on])]

    df['local_min+-' + str(period)] = df[_on][
      (df[_on].shift(period) > df[_on]) &
      (df[_on].shift(-period) > df[_on])]
    df['local_min+-' + str(period)].fillna(method='ffill', inplace=True)
    df['local_max+-' + str(period)].fillna(method='ffill', inplace=True)
    return df

df = explore_minima(df,1)
#df.head(20)

In [109]:
'''df_ta = add_all_ta_features(
    df, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)
print(df_ta.shape)
df_ta'''

'df_ta = add_all_ta_features(\n    df, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)\nprint(df_ta.shape)\ndf_ta'

In [144]:
df = TFs_to_predict(df, 'Close', timeframe, 10)
df

,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,year,...,1dayClose1,1dayClose2,1dayClose3,1dayClose4,1dayClose5,1dayClose6,1dayClose7,1dayClose8,1dayClose9,1dayClose10
0,34049.15,37850.00,32380.00,37371.38,124477.914938,4.322877e+09,2514289,63981.038306,2.222873e+09,2021,...,39144.50,36742.22,35994.98,35828.61,36631.27,35891.49,35468.23,30850.13,32945.17,32078.00
1,37371.38,40100.00,36701.23,39144.50,102950.389421,3.974827e+09,2297466,51264.467092,1.979443e+09,2021,...,36742.22,35994.98,35828.61,36631.27,35891.49,35468.23,30850.13,32945.17,32078.00,32259.90
2,39145.21,39747.76,34408.00,36742.22,118300.920916,4.371589e+09,2560801,57738.034220,2.133724e+09,2021,...,35994.98,35828.61,36631.27,35891.49,35468.23,30850.13,32945.17,32078.00,32259.90,32254.20
3,36737.43,37950.00,35357.80,35994.98,86348.431508,3.178840e+09,2066363,42420.423736,1.563160e+09,2021,...,35828.61,36631.27,35891.49,35468.23,30850.13,32945.17,32078.00,32259.90,32254.20,32467.77
4,35994.98,36852.50,33850.00,35828.61,80157.727384,2.843103e+09,1860642,39513.638561,1.402649e+09,2021,...,36631.27,35891.49,35468.23,30850.13,32945.17,32078.00,32259.90,32254.20,32467.77,30366.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,30293.93,30670.51,28866.35,29109.15,63901.499320,1.914306e+09,1328317,30476.668110,9.138140e+08,2022,...,29654.58,29542.15,29201.35,29005.51,NaN,NaN,NaN,NaN,NaN,NaN
496,29109.14,29845.86,28669.00,29654.58,59442.960360,1.740258e+09,946421,29480.076740,8.635167e+08,2022,...,29542.15,29201.35,29005.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,29654.58,30223.74,29294.21,29542.15,59537.386590,1.771708e+09,985440,28846.965890,8.585873e+08,2022,...,29201.35,29005.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,29542.14,29886.64,28019.56,29201.35,94581.654630,2.759055e+09,1431112,45206.854160,1.319426e+09,2022,...,29005.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#plt_df(df, ['Open','Close','High', 'Low'], 100)

In [132]:
scaler_features, train_scaled = standard_scaler(df.values)
df_scaled = pd.DataFrame(train_scaled)
df_scaled.columns = df.columns

In [133]:
days_to_predict = ["dayClose1","dayClose2","dayClose3","dayClose4","dayClose5","dayClose6","dayClose7","dayClose8","dayClose9","dayClose10"]
df_new = labelize_output(df, days_to_predict)
df_new.columns

KeyError: 'dayClose1'

In [135]:
df_new = df_scaled
df_new.dropna(inplace=True)
df_new.isnull().sum()
X_train, X_test, Y_train, Y_test = df_train_test_split(df_new, [x for x in df_new.columns if x.startswith('1dayClose')], pct_train=0.9)


(268, 173) (268, 10)


In [98]:
parameters = {
    'learning_rate': [0.05, 0.1, 0.2, 0.3],
    'gamma': [0],
    'max_depth': [3, 5, 7],
    'min_child_weight': [5, 10, 15, 20, 25],
    'subsample': [1],
    'colsample_bytree': [0.8, 1],
    'n_estimators': [100],
    'objective': ['binary:logistic'],
    'eval_metric': ['logloss'],
    'use_label_encoder': [False]
}

param = {
    'learning_rate': 0.1,
    'gamma': 0,
    'max_depth': 5,
    'min_child_weight': 15,
    'subsample': 1,
    'colsample_bytree': 1,
    'n_estimators': 100,
    #'objective': 'binary:logistic',
    'objective': 'reg:squarederror',
    'eval_metric': 'error'

}

In [99]:
# call xgb
model_type='xgb'
model_base = model_selection(model_type, param,'regression')
model_ = MultiOutputRegressor(model_base)

In [100]:
list_param_tuning = [dict(zip(parameters.keys(), el))
                         for el in itertools.product(*parameters.values())]

In [ ]:
X_train.head()

In [ ]:
Y_train.head()

In [ ]:
model_.fit(X_train,Y_train)    

In [ ]:
yhat = model_.predict(X_test)

In [ ]:
yhat[:,0]

In [ ]:
yhat[:,0].shape,Y_test["1dayClose2"].shape[0]

In [ ]:
plt.plot(range(Y_test["1dayClose1"].shape[0]), Y_test["1dayClose1"], color ='red')
plt.plot(range(Y_test["1dayClose1"].shape[0]), yhat[:,0])
plt.show()

In [63]:
col_target = ["1dayClose1","1dayClose2","1dayClose3","1dayClose4","1dayClose5","1dayClose6","1dayClose7","1dayClose8","1dayClose9","1dayClose10"]


In [145]:
df_new = df

# Applico PPscore per ogni target
for i, target in enumerate(col_target):
    cols_to_drop = list(set(col_target) - set([target]))
    data_pps = df_new.drop(columns=cols_to_drop)
    pps_pred = pps.predictors(data_pps, target)
    # Estraggo i valori del pps per ogni colonna
    if i == 0:
        data_pps_final = pps_pred[['x', 'ppscore']].copy()
    else:
        data_pps_final = data_pps_final.merge(pps_pred[['x', 'ppscore']], on=['x'], how='outer', suffixes=('_y' + str(i), '_y' + str(i+1)))

data_pps_final

,x,ppscore_y1,ppscore_y2,ppscore_y3,ppscore_y4,ppscore_y5,ppscore_y6,ppscore_y7,ppscore_y8,ppscore_y9,ppscore_y10
0,MIN5_Close,0.754231,0.709838,0.678501,0.665065,0.637144,0.635331,0.643622,0.645149,0.627125,0.636490
1,Close,0.752903,0.664704,0.580847,0.507874,0.440393,0.380955,0.369992,0.350437,0.285575,0.249112
2,MAX5_Close,0.752031,0.741097,0.709555,0.707190,0.667103,0.660708,0.681226,0.693886,0.675058,0.648185
3,MAX7_Open,0.751046,0.725741,0.718480,0.694509,0.672064,0.674768,0.676339,0.652348,0.654274,0.642695
4,MIN5_High,0.742998,0.700332,0.661848,0.667179,0.649736,0.644929,0.611329,0.595981,0.600920,0.602813
...,...,...,...,...,...,...,...,...,...,...,...
168,STD21_Low,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
169,STD28_Open,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
170,STD28_Close,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
171,STD28_High,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [146]:
data_pps_final = data_pps_final.fillna(0) # per evitare nan
data_pps_final['tot_pps'] = data_pps_final[[pps_col for pps_col in data_pps_final.columns.tolist() if 'ppscore' in pps_col]].mean(axis=1)
data_pps_final = data_pps_final.sort_values(by=['tot_pps'], ascending=False)
data_pps_final

,x,ppscore_y1,ppscore_y2,ppscore_y3,ppscore_y4,ppscore_y5,ppscore_y6,ppscore_y7,ppscore_y8,ppscore_y9,ppscore_y10,tot_pps
6,MAX7_Close,0.737784,0.739201,0.714400,0.703150,0.700022,0.686309,0.680803,0.667767,0.675137,0.645583,0.695016
2,MAX5_Close,0.752031,0.741097,0.709555,0.707190,0.667103,0.660708,0.681226,0.693886,0.675058,0.648185,0.693604
12,MAX10_Close,0.730697,0.726718,0.717478,0.705239,0.699757,0.686885,0.665555,0.669022,0.662387,0.651589,0.691533
5,MIN7_Close,0.738647,0.730240,0.687685,0.696399,0.693777,0.679924,0.667071,0.674437,0.665167,0.635953,0.686930
10,MAX10_Open,0.733193,0.717986,0.709119,0.708778,0.679798,0.689297,0.673797,0.661499,0.655403,0.635804,0.686467
...,...,...,...,...,...,...,...,...,...,...,...,...
140,SMA223_High,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
141,STD5_Open,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
142,STD5_Close,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
143,STD5_High,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [147]:
data_pps_clean =  data_pps_final.loc[~(data_pps_final['tot_pps']<=0.5)]
data_pps_clean = data_pps_clean.sort_values(by=['tot_pps'], ascending=False)
data_pps_clean.head(20)

,x,ppscore_y1,ppscore_y2,ppscore_y3,ppscore_y4,ppscore_y5,ppscore_y6,ppscore_y7,ppscore_y8,ppscore_y9,ppscore_y10,tot_pps
6,MAX7_Close,0.737784,0.739201,0.714400,0.703150,0.700022,0.686309,0.680803,0.667767,0.675137,0.645583,0.695016
2,MAX5_Close,0.752031,0.741097,0.709555,0.707190,0.667103,0.660708,0.681226,0.693886,0.675058,0.648185,0.693604
12,MAX10_Close,0.730697,0.726718,0.717478,0.705239,0.699757,0.686885,0.665555,0.669022,0.662387,0.651589,0.691533
5,MIN7_Close,0.738647,0.730240,0.687685,0.696399,0.693777,0.679924,0.667071,0.674437,0.665167,0.635953,0.686930
10,MAX10_Open,0.733193,0.717986,0.709119,0.708778,0.679798,0.689297,0.673797,0.661499,0.655403,0.635804,0.686467
3,MAX7_Open,0.751046,0.725741,0.718480,0.694509,0.672064,0.674768,0.676339,0.652348,0.654274,0.642695,0.686227
14,MAX5_Open,0.729142,0.722172,0.707317,0.680752,0.674617,0.660685,0.674494,0.665682,0.653419,0.636390,0.680467
21,MAX14_Close,0.720361,0.713580,0.697034,0.687287,0.689415,0.669218,0.654120,0.662706,0.647173,0.659608,0.680050
15,MAX7_Low,0.728487,0.708565,0.691699,0.669246,0.690016,0.683462,0.653508,0.665980,0.660804,0.632501,0.678427
9,MIN7_Open,0.733501,0.710304,0.697478,0.689981,0.681984,0.663745,0.674462,0.639887,0.650430,0.635829,0.677760


In [ ]:
print(len(model_.estimators_))
model_.estimators_[0].feature_importances_

In [ ]:
plot_importance(model_.estimators_[0])
plt.show()

In [ ]:
data = list(zip(df_new.columns, model_.estimators_[0].feature_importances_))
new_df = pd.DataFrame(data, columns =['Feature', 'Importance'])
new_df

In [ ]:

            # Predizione su train e val
            val_pred_0 = model_.predict(val_features_0)
            train_pred_0 = model_.predict(train_features_0)
            
            val_pred_prob_0 = np.transpose([pred[:, 1] for pred in model_.predict_proba(val_features_0)])
            train_pred_prob_0 = np.transpose([pred[:, 1] for pred in model_.predict_proba(train_features_0)])

            # calcolo score per train e val
            scores_val_fbeta.append(
                fbeta_score(val_labels_0, val_pred_0, beta=0.5, average='micro'))
            scores_train_fbeta.append(
                fbeta_score(train_labels_0, train_pred_0, beta=0.5, average='micro'))
            scores_val_log_loss.append(log_loss(
                val_labels_0, val_pred_prob_0))
            scores_train_log_loss.append(log_loss(
                train_labels_0, train_pred_prob_0))


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, pacf

def get_relevant_lag(data, target='Open', n_lags=40, fixed_lag=[1, 2, 3]):
    
    data_ = data.copy()
    
    # Aggrego il dato per ogni time frame
    #ts = data_.groupby(time_frame)[target].sum()[:-1]

    # Calcolo PACF
    lag, ci = pacf(data_[target], nlags=n_lags, alpha=0.05)
    #plot_pacf(ts, lags=n_lags, alpha=0.05)
    #ic(lag,ci)
    # Calcolo il limite (area blu in grafico pacf)      
    boundaries = np.abs(np.array([ci - lag for lag, ci in zip(lag, ci)])[:, 1])
    
    # estraggo i lags maggiori dei limiti
    imp_lags = np.where(np.abs(lag) > boundaries)[0]
    
    imp_lags = list(set(imp_lags) - set([0]+fixed_lag))
    
    return imp_lags

In [ ]:
x = get_relevant_lag(df_new)
x

In [ ]:
plot_pacf(df_new["Open"], lags=40)
pyplot.show()

In [ ]:
df = pd.DataFrame()
df["x"] = np.random.uniform(-2, 2, 1_000_000)
df["error"] = np.random.uniform(-0.5, 0.5, 1_000_000)
df["y"] = df["x"] * df["x"] + df["error"]

In [ ]:
pps.score(df, "x", "y")

In [ ]:
pps.predictors(df, "y")
